# Google ColabでのYOLOv8モデルのトレーニング

このノートブックは、Google Driveに保存されたカスタムデータセットを使用して、Google Colab上でYOLOv8モデルをトレーニングするためのものです。
以下の内容をカバーします：
1. Google Driveのマウント。
2. `ultralytics`ライブラリのインストール。
3. データセットとプロジェクト出力のパス設定。
4. YOLOv8モデルのトレーニング。
5. トレーニング済みモデルと結果の保存場所に関する情報。

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ultralyticsライブラリをインストール
!pip install ultralytics

## Gitのセットアップ (オプション)

Colabから直接Gitリポジトリにコミットやプッシュを行いたい場合、以下のセルでGitのユーザー名とメールアドレスを設定します。
これは、コミット履歴に記録される作成者情報となります。

**注意:** ここで設定するメールアドレスやユーザー名は公開される可能性があるため、取り扱いに注意してください。

In [ ]:
# Gitのユーザー名とメールアドレスを設定
# 以下の "Your Name" と "youremail@example.com" をご自身のものに置き換えてください。
!git config --global user.name "Your Name" # ★★★ あなたの名前に置き換えてください ★★★
!git config --global user.email "youremail@example.com" # ★★★ あなたのメールアドレスに置き換えてください ★★★

print("Gitのユーザー名とメールアドレスが設定されました。")
!git config --global --list # 設定内容の確認

## パスの設定

データセットの`data.yaml`ファイルと、トレーニング結果（例: モデルの重み、ログ）を保存するディレクトリのパスを定義します。

## Google Drive上のデータセットを展開 (オプション)

もしデータセットがGoogle Drive上でZIPファイルとして保存されている場合、以下のセルを実行してColabのローカルストレージに展開できます。
これにより、トレーニング中のファイルアクセスが高速になることがあります。

**注意:**
- ZIPファイルへのパス (`zip_file_path_on_drive`) と展開先のディレクトリ (`colab_dataset_dir`) をご自身の環境に合わせて設定してください。
- データセットが大きい場合、展開に時間がかかることがあります。
- Colabのセッションが終了すると、展開されたデータは削除されます。再度ノートブックを実行する際には、このセルも再実行する必要があります。
- このセルを実行した場合、次の「パスの設定」セルで `dataset_yaml_path` が、Colab上に展開されたデータセット内の `data.yaml` を指すように必ず更新してください。

In [ ]:
import zipfile
import os

# --- ★★★ ユーザー設定項目 ★★★ ---
# Google Drive上のZIPファイルのパス
# 例: '/content/drive/MyDrive/datasets/my_dataset.zip'
zip_file_path_on_drive = '/content/drive/MyDrive/your_dataset.zip'  # ★★★ あなたのZIPファイルのパスに置き換えてください ★★★

# Colab上にデータセットを展開するディレクトリ
# 通常、'/content/' 以下に作成します。
colab_dataset_dir = '/content/datasets' # ★★★ 必要であれば変更してください ★★★

# --- ★★★ 設定項目終了 ★★★ ---

# 展開先ディレクトリを作成
if not os.path.exists(colab_dataset_dir):
    os.makedirs(colab_dataset_dir)
    print(f"展開先ディレクトリを作成しました: {colab_dataset_dir}")
else:
    print(f"展開先ディレクトリは既に存在します: {colab_dataset_dir}")

if os.path.exists(zip_file_path_on_drive):
    print(f"ZIPファイルが見つかりました: {zip_file_path_on_drive}")
    print(f"ファイルを {colab_dataset_dir} に展開しています...")
    try:
        with zipfile.ZipFile(zip_file_path_on_drive, 'r') as zip_ref:
            zip_ref.extractall(colab_dataset_dir)
        print(f"データセットの展開が完了しました。展開先: {colab_dataset_dir}")
        
        print("\n--- 展開後の確認と次のステップ --- ")
        print(f"展開先ディレクトリ ({colab_dataset_dir}) の内容:")
        extracted_items = os.listdir(colab_dataset_dir)
        if extracted_items:
            for item in extracted_items:
                print(f"- {item}")
        else:
            print("(ディレクトリは空か、アクセスできませんでした)")

        print(f"\n重要: 次の「パスの設定」セルで、`dataset_yaml_path` が、")
        print(f"このColab上の展開先ディレクトリ ({colab_dataset_dir}) 内の `data.yaml` を指すように更新してください。")
        print(f"例えば、もしZIP展開後に '{colab_dataset_dir}/your_dataset_main_folder/data.yaml' のような構造になる場合、")
        print(f"dataset_yaml_path = '{os.path.join(colab_dataset_dir, "your_dataset_main_folder", "data.yaml")}' のように設定します。")
        print("ZIPファイル内のフォルダ構造を確認し、適切にパスを修正してください。")

    except zipfile.BadZipFile:
        print(f"エラー: {zip_file_path_on_drive} は有効なZIPファイルではありません。")
    except Exception as e:
        print(f"展開中にエラーが発生しました: {e}")
else:
    print(f"エラー: ZIPファイルが見つかりません: {zip_file_path_on_drive}")
    print("Google Drive上のパスを確認してください。")
    print("もしZIP展開機能を使用しない場合は、このセルをスキップして問題ありません。")

In [ ]:
import os

# --- 重要: これらのパスをGoogle Driveの構造に合わせて設定してください ---

# Google Drive上のデータセットのdata.yamlファイルへのパス
# 例: '/content/drive/MyDrive/datasets/tennis_data/data.yaml'
dataset_yaml_path = '/content/drive/MyDrive/your_dataset_folder/data.yaml' # ★★★ ここを修正してください ★★★

# トレーニング結果（重み、ログなど）を保存するGoogle Drive上のディレクトリ
# 例: '/content/drive/MyDrive/YOLOv8_Tennis_Project'
project_output_dir = '/content/drive/MyDrive/YOLOv8_Training_Outputs' # ★★★ ここを修正してください ★★★

# この特定のトレーニング実行の名前（project_output_dirのサブディレクトリとして作成されます）
experiment_name = 'tennis_detection_run1'

# --- パス設定終了 ---

# プロジェクト出力ディレクトリが存在しない場合は作成
os.makedirs(project_output_dir, exist_ok=True)

print(f"データセットのYAMLパス: {dataset_yaml_path}")
print(f"プロジェクト出力ディレクトリ: {project_output_dir}")
print(f"実験名: {experiment_name}")

# データセットのYAMLファイルが存在するか確認
if not os.path.exists(dataset_yaml_path):
    print(f"エラー: データセットのYAMLファイルが見つかりません: {dataset_yaml_path}")
    print("パスを再確認してください。")
else:
    print("データセットのYAMLファイルが見つかりました。")

## YOLOv8モデルの初期化とトレーニング

In [ ]:
from ultralytics import YOLO

# 事前学習済みのYOLOv8モデルをロード
# 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt' などのサイズを選択可能
# 初めての場合は 'yolov8n.pt' (Nano) または 'yolov8s.pt' (Small) を推奨
model_name = 'yolov8s.pt'
model = YOLO(model_name)

# トレーニングのハイパーパラメータを定義
num_epochs = 100  # トレーニングエポック数 (例: 50, 100, 200)
batch_size = 16   # バッチサイズ (GPUメモリに応じて調整: 例 8, 16, 32)
img_size = 640    # 入力画像サイズ (例: 640, 1280)

print(f"モデルのトレーニングを開始します: {model_name}")
print(f"エポック数: {num_epochs}, バッチサイズ: {batch_size}, 画像サイズ: {img_size}")

# トレーニングを開始
try:
    results = model.train(
        data=dataset_yaml_path,
        epochs=num_epochs,
        imgsz=img_size,
        batch=batch_size,
        project=project_output_dir, # すべての実行結果を保存するメインディレクトリ
        name=experiment_name,       # この特定の実行のサブディレクトリ
        exist_ok=True,              # experiment_nameディレクトリが既に存在する場合は上書きを許可
        # device=0,                 # GPUを使用 (最初のGPUは0, CPUを使用する場合は 'cpu')
                                    # Colabでは通常、自動的にGPUが割り当てられます。
        # workers=8,                # データローダーのワーカー数 (ColabのCPUに応じて調整)
        # patience=30,              # 早期終了の忍耐値 (例: 30エポック後に改善がない場合に停止)
        # lr0=0.01,                 # 初期学習率
        # optimizer='AdamW',        # オプティマイザ (例: 'SGD', 'Adam', 'AdamW')
    )
    print("\nトレーニングが正常に完了しました！")
    print(f"結果、ログ、モデルの重みは次の場所に保存されました: {os.path.join(project_output_dir, experiment_name)}")
except Exception as e:
    print(f"\nトレーニング中にエラーが発生しました: {e}")

## トレーニング結果と保存されたモデル

トレーニング後、結果（メトリクス、混同行列、モデルの重みなど）は指定したディレクトリに保存されます：
`{project_output_dir}/{experiment_name}`

確認すべき主なファイル：
- **重み:** `weights`サブディレクトリ内 (例: `best.pt`, `last.pt`)
  - `best.pt`: 最良の検証メトリクス（通常はmAP50-95）を達成したモデルの重み。このモデルを推論に使用するのが一般的です。
  - `last.pt`: トレーニングの最終エポックのモデルの重み。
- **結果CSV:** `results.csv`にはエポックごとのメトリクスの概要が含まれています。
- **プロット:** 混同行列、P-R曲線などのさまざまなプロット (例: `confusion_matrix.png`, `PR_curve.png`)

これらのファイルはGoogle Driveからダウンロードできます。

In [ ]:
# 実験の重みディレクトリ内のファイルをリストアップする例
weights_dir = os.path.join(project_output_dir, experiment_name, 'weights')
if os.path.exists(weights_dir):
    print(f"\n重みディレクトリ内のファイル ({weights_dir}):")
    for f_name in os.listdir(weights_dir):
        print(f"- {f_name}")
else:
    print(f"\n重みディレクトリが見つかりません: {weights_dir}")
    print("トレーニングが完了していないか、エラーが発生した可能性があります。")

# トレーニング済みモデルの最良の重みへのパス（後で推論に使用可能）
best_model_path = os.path.join(weights_dir, 'best.pt')
print(f"\n最良のモデルへのパス: {best_model_path}")
if os.path.exists(best_model_path):
    print("最良のモデルファイル (best.pt) が見つかりました。")
else:
    print("最良のモデルファイル (best.pt) が見つかりません。トレーニングが失敗したか、生成されなかった可能性があります。")